## 2 Cleaning numerical data

In [0]:
dfcopy.select_dtypes(include=[float]).head()

### 2.1 Dealing with outliers

In [0]:
cols_100g = list(dfcopy.columns[dfcopy.columns.str.contains('_100g')])
df_100g = dfcopy.loc[:,cols_100g].astype(float).copy()
df_cat_nutri = pd.concat([dfcopy.loc[:,'nutriscore'].astype(float),
                          dfcopy.loc[:,['nutrigrade','pnns_gp1',
                                        'main_categ_en']]]).copy()
df_cat_nutri.columns=['nutriscore','nutrigrade','pnns_gp1', 'main_categ_en']
df_cat_nutri

In [0]:
dfcopy.loc[:,'nutriscore'].isna().sum(), df_cat_nutri.loc[:,'nutriscore'].isna().sum()

#### 2.1.1 columns about nutriscores

In [0]:
speak("attention savastoppé")

In [0]:
# Checking if ther are rows where 'nutriscore' is different from 'nutriscore-fr'
cond = dfcopy['nutriscore']==dfcopy['nutriscore-fr']
dfcopy.dropna(subset=['nutriscore', 'nutriscore-fr'])[~cond].shape[0]

Column 'nutriscore' is not different from 'nutriscore-fr'.
 We merge the columns and delete 'nutriscore-fr'




In [0]:
dfcopy['nutriscore'] = dfcopy["nutriscore-fr"].combine_first(dfcopy["nutriscore"])
dfcopy.drop(columns=["nutriscore-fr"], inplace=True)

The nutrigrade is assigned to a product after consideration of :
- its nutriscore
- whether it's solid food or beverage 

In [0]:
dfcopy.groupby('quantity_type').size()

In [0]:
# dfcopy.boxplot(by=['nutrigrade', 'quantity_type'])

In [0]:
gb = dfcopy.groupby(by=['nutrigrade', 'quantity_type'])

"Outil_Calcul_Nutri-Score_SpF-030120" :
https://www.santepubliquefrance.fr/determinants-de-sante/nutrition-et-activite-physique/articles/nutri-score (tableur excel)

1823 products are not identified as liquid/beverage or solid/food.
Let's see if we can guess from the categories columns. 

#### 2.2.2 Columns about energy

en

In [0]:
# group1: all nan, can't do anything for the moment
df_gp1 = df_100g.loc[gb.groups.get((False, False, False))]
df_gp1.shape

In [0]:
# group2: group to send to the model -> is it kj or kcal ?
df_gp2 = df_100g.loc[gb.groups.get((True, False, False))]
df_gp2.shape

In [0]:
# group3: group where e is j
df_gp3 = df_100g.loc[gb.groups.get((True, False, True))]
print(df_gp3.shape)
m_id3 = df_gp3.apply(lambda x: x['energy-kj_100g']==x['energy_100g'], axis=1)
df_gp3[~m_id3].shape # cas où ils sont différents (2) : prévoir remplacement (fonction)

In [0]:
# group4: group where e is c 
df_gp4 = df_100g.loc[gb.groups.get((True, True, False))]
print(df_gp4.shape)
m_id4 = df_gp4.apply(lambda x: x['energy-kcal_100g']==x['energy_100g'], axis=1)
df_gp4[~m_id4].shape # cas où ils sont différents (0) : prévoir remplacement (fonction)

In [0]:
# group5: group where e, c and j are both filled : are they the same ? which is the correct one ? to be submitted to the model
df_gp5 = df_100g.loc[gb.groups.get((True, True, True))]
print(df_gp5.shape)
f_e_eq_c = lambda x: x['energy-kcal_100g']==x['energy_100g']
f_e_eq_j = lambda x: x['energy-kj_100g']==x['energy_100g']
m_id_c5 = df_gp5.apply(f_e_eq_c, axis=1)
m_id_j5 = df_gp5.apply(f_e_eq_j, axis=1)
df_gp5_A = df_gp5[(m_id_c5&m_id_j5)]
df_gp5_B = df_gp5[(~m_id_c5)&(~m_id_j5)]
df_gp5_C = df_gp5[(~(m_id_c5&m_id_j5))&(~((~m_id_c5)&(~m_id_j5)))]
df_gp5_C_c = df_gp5_C[m_id_c5]
df_gp5_C_j = df_gp5_C[m_id_j5]
print(df_gp5_A.shape) # cas où j, c et e sont égaux (env 2 000)
print(df_gp5_B.shape) # cas où e, c et j sont tous différents (0)
print(df_gp5_C.shape) # cas où j, e et c ne sont pas tous égaux mais pas tous différents (au moins un couple égaux) (env 20 000)
print(df_gp5_C_c.shape)
print(df_gp5_C_j.shape)

In [0]:
'''données False True True inexistantes 
préparer un modèle pour détecter la catégorie de calorie (True False False)
éliminer données False False False
d'abord modifier les données de type True True True en : True True False ou True, False true
puis garder les données de type
True, False, True ou
True, True, False
------------------
les données auront 4 colonnes et 1 target
target à créer : kcal 0 ou 1
mettre en place un modèle random forest

In [0]:
# Gathering rows and columns to form a dataset
sel_en_cols = ['energy_100g','fat_100g', 'carbohydrates_100g', 'proteins_100g',
            'fiber_100g', 'alcohol_100g', 'pnns_gp1', 'main_categ_en', 'nutriscore', 'nutrigrade']
train_set = pd.concat([df_gp3[m_id3], df_gp3[m_id3]])[sel_en_cols]